In [1]:
#First, the point cloud has to be loaded and then read.
#Its probably way too big to work with from the start, so I think I will use voxel downsampling
#But first, imports

import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import os   
from pyntcloud import PyntCloud

#At first im going to try to just load the pointcloud 
#It runs insanely slow, so I will try to downsample it

initialCloud =  o3d.io.read_point_cloud(os.path.join(os.getcwd(), "data", "croppedCloud.ply"))




Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
Optional step: #Downsampling the cloud first because it takes forever to run

#initialCloud = initialCloud.voxel_down_sample(0.3)

In [ ]:
#print("Number of points after downsampling ", len(initialCloud.points))

In [3]:
#Making a voxelgrid from the point cloud
voxel_size = 3
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(initialCloud, voxel_size)



In [ ]:
o3d.visualization.draw_geometries([voxel_grid])

In [14]:
#Extracting center points so we can calculate PCA for the points within each voxel

voxel_center_coordinates= []

for i in voxel_grid.get_voxels():
    voxel_center_coordinates.append(voxel_grid.get_voxel_center_coordinate(i.grid_index))



In [5]:
#Now we have to extract the points from the point cloud that are within each voxel
#We can do this with the get_voxel_coordinates function

print(len(voxel_grid.get_voxels()))
    



29528


In [ ]:
#Now we have to extract the points from the point cloud that are within each voxel
voxel_size = 0.85
center = voxel_center_coordinates[56]

minboundx = center[0] - voxel_size/2
minboundy = center[1] - voxel_size/2
minboundz = center[2] - voxel_size/2

maxboundx = center[0] + voxel_size/2
maxboundy = center[1] + voxel_size/2
maxboundz = center[2] + voxel_size/2

voxel_extracted_points =initialCloud.crop(o3d.geometry.AxisAlignedBoundingBox(min_bound=(minboundx, minboundy, minboundz), max_bound=(maxboundx, maxboundy, maxboundz)))

In [ ]:
print(np.asarray(voxel_extracted_points.points))

o3d.visualization.draw_geometries([voxel_extracted_points])


In [ ]:
#Calculating PCA for the points within the extracted voxel
from sklearn.decomposition import PCA

points = np.asarray(voxel_extracted_points.points)

# Assuming 'points' contains the 3D coordinates of your point cloud
pca = PCA(n_components=3)
pca.fit(points)

# Extract the eigenvalues and eigenvectors
eigenvalues = pca.explained_variance_
eigenvectors = pca.components_



In [ ]:
print(eigenvalues)
planarity = (eigenvalues[1] - eigenvalues[2]) / eigenvalues[1]

print(planarity)

In [16]:

from sklearn.decomposition import PCA
from tqdm.auto import tqdm
#Now we have to extract the points from the point cloud that are within each voxel
voxel_size = 3


linearity =[]
planarity = []
scattering = []
omnivariance = []
sum_eigenvalues = []
anisotropy = []
eigentropy = []
change_of_curvature = []

#Run this to just run it on the first 100 voxels
#voxel_center_coordinates = voxel_center_coordinates[:100]

for center in tqdm(voxel_center_coordinates):
    minboundx = center[0] - voxel_size/2
    minboundy = center[1] - voxel_size/2
    minboundz = center[2] - voxel_size/2

    maxboundx = center[0] + voxel_size/2
    maxboundy = center[1] + voxel_size/2
    maxboundz = center[2] + voxel_size/2

    voxel_extracted_points =initialCloud.crop(o3d.geometry.AxisAlignedBoundingBox(min_bound=(minboundx, minboundy, minboundz), max_bound=(maxboundx, maxboundy, maxboundz)))
    #print("Number of points in voxel ", len(voxel_extracted_points.points))
    
    points = np.asarray(voxel_extracted_points.points)

    #Checking if there are enough points in the voxel to calculate PCA
    
    if len(points) < 15:
        linearity.append(0)
        planarity.append(0)
        scattering.append(0)
        omnivariance.append(0)
        sum_eigenvalues.append(0)
        anisotropy.append(0)
        eigentropy.append(0)
        change_of_curvature.append(0)
        continue
    
    pca = PCA(n_components=3)
    pca.fit(points)

    # Extract the eigenvalues and eigenvectors
    eigenvalues = pca.explained_variance_
    eigenvectors = pca.components_
    
    #Calculating the features
    l = (eigenvalues[0] - eigenvalues[1]) / eigenvalues[0] 
    p = (eigenvalues[1] - eigenvalues[2]) / eigenvalues[0]
    s = eigenvalues[2] / eigenvalues[0]
    o = (eigenvalues[0]*eigenvalues[1]*eigenvalues[2])**(1/3)
    s_e = eigenvalues[0] + eigenvalues[1] + eigenvalues[2]
    a = (eigenvalues[0] - eigenvalues[2]) / eigenvalues[0]
    e = (-1)*(eigenvalues[0]*np.log(eigenvalues[0]) + eigenvalues[1]*np.log(eigenvalues[1]) + eigenvalues[2]*np.log(eigenvalues[2]))
    c_o_c = eigenvalues[2]/(eigenvalues[0]+eigenvalues[1]+eigenvalues[2])
    
    linearity.append(l)
    planarity.append(p)
    scattering.append(s)
    omnivariance.append(o)
    sum_eigenvalues.append(s_e)
    anisotropy.append(a)
    eigentropy.append(e)
    change_of_curvature.append(c_o_c)
   
    

  0%|          | 0/29528 [00:00<?, ?it/s]

In [13]:
print(len(voxel_center_coordinates))

100


In [ ]:
print(curvatures)